In [113]:
import math
import sys
import os
import pandas as pd
import numpy as np
import time
import json
from multiprocessing import Pool


sdk_path = '/home/xz/visenze/visearch-sdk-python/'
sys.path.append(sdk_path)

from visearch.client import ViSearchAPI

In [114]:
is_test = False
# Initialize ViSearch
#dev for gender
access_key = '38317a4df753abbc2bfffaa5229c2c78'
secret_key = '24e8e074336d1506656f9556fb38b7f2'
api = ViSearchAPI(access_key, secret_key)

if is_test:
    #test for gender
    access_key = '7c116332a99b0c7c23c03ab96b35388e'
    secret_key = '00cf5e60a4fee9d66120e9bd0e86281c'
    api = ViSearchAPI(access_key, secret_key)

In [47]:
def search(item):
    start = time.time()
    for retry in range(10):
        response = api.uploadsearch(
            image_url=item['S3 URL'],
            score=True,
            fl=['category','category_original'],
            limit=1000,
            detection='all',
            vtt_source='visenze_admin',
            )
        
        duration = time.time() - start
        # Add delay to make sure each image takes at least 1s
        if duration < 1.2:
            time.sleep(1.2 - duration)

        if response['status'] != 'OK':
            print(response['error'])
            print(' --> retry ({})'.format(retry))
            continue

        # Remove some useless stuffs
        useless_items = [
            'product_types_list',
            'page',
            'reqid',
            'im_id',
            'method',
            'total'
            ]

        # Append query info
        response['query_id'] = item['Image Id']
        response['query_url'] = item['S3 URL']
        response['query_category'] = item['category']
        response['query_category_original'] = item['Category']

        for name in useless_items:
            if name in response:
                del response[name]

        response['latency'] = duration
        break

    return response
#-----------------------

In [48]:
data_folder = os.path.join(os.getcwd(), 'data')
#load dev/test query set and get response
dev_query_set = pd.read_csv(os.path.join(data_folder, 'dev_query_w_predict.csv'))
test_query_set = pd.read_csv(os.path.join(data_folder, 'test_query_w_predict.csv'))

def get_api_response(query_set, output_name):
    pool = Pool(processes=12)

    results = []
    success = 0
    ret = [pool.apply_async(search, (item,)) for i,item in query_set.iterrows()]

    for r in ret:
        res = r.get(timeout=60)
        if res['status'] == 'OK':
            success += 1
        else:
            print(res['error'])

        results += [res]

    print('Successful: {}/{}'.format(success, len(query_set)))

    # Dump results to file
    with open(os.path.join(data_folder, output_name), 'w') as f:
        json.dump(results, f, indent=2)
    return

# get_api_response(dev_query_set, 'dev_raw_response')
get_api_response(test_query_set, 'test_raw_response')

Successful: 1635/1635


Process ForkPoolWorker-1:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-9:
Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-2:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/xz/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xz/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xz/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xz/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xz/anaconda3/lib/python3.7/multiprocessing/process.py", lin

KeyboardInterrupt


In [49]:
with open(os.path.join(data_folder, 'dev_raw_response'), 'r') as f:
    dev_raw_response = json.load(f)
with open(os.path.join(data_folder, 'test_raw_response'), 'r') as f:
    test_raw_response = json.load(f)

In [50]:
#load dev query and ground truth
category_gt_dev = pd.read_csv(os.path.join(data_folder, 'category_gt_dev.csv'))
category_query_dev = pd.read_csv(os.path.join(data_folder, 'category_query_dev.csv'))

category_gt_test = pd.read_csv(os.path.join(data_folder, 'category_gt_test.csv'))
category_query_test = pd.read_csv(os.path.join(data_folder, 'category_query_test.csv'))

In [68]:
def rank_score(result, category, w=0.0):
    score = result['score']
    category_score = 1 if category == result['value_map']['category'] else 0
    score += w * category_score
    return score

NDCG_weights = [1 / math.log2(i + 2) for i in range(10)]
def cal_conformity_NDCG(raw_response, rank_score, query_set, gt_set, w=0.0):
    conformity = []
    category_DCG = []
    NDCG = []
    for q in raw_response:
        results = q['result']
        category_ori = q['query_category_original']
        category = q['query_category']
        
        #rerank based on customized score in decreasing order
        results = sorted(results, key=lambda x: -rank_score(x, category, w))
        #calculate per query conformity
        topk = min(10, len(results))
        match_cnt = 0
        cur_category_DCG = 0
        for i in range(topk):
            if results[i]['value_map']['category_original'] == category_ori:
                match_cnt += 1
                cur_category_DCG += NDCG_weights[i]
        conformity.append((match_cnt + 0.0) / topk)
        category_DCG.append(cur_category_DCG)
       
        query_url = query_set.loc[query_set['S3 URL'] == q['query_url']]['Query Set URL'].iloc[0]
        gt = gt_set.loc[gt_set['Query Set URL'] == query_url]
        gt = gt.sort_values(by=['Score'], ascending=False)
        gt_map = dict()

        IDCG = 0
        gt_len = min(10, len(gt))
        for i in range(gt_len):
            row = gt.iloc[i]
            IDCG += row['Score'] * NDCG_weights[i]
            gt_map[row['Im_Name']] = row['Score']

        DCG = 0
        for i in range(topk):
            if results[i]['im_name'] in gt_map:
                DCG += gt_map[results[i]['im_name']] * NDCG_weights[i]
        NDCG.append(DCG / IDCG)
              
    return (sum(conformity) / len(conformity), sum(category_DCG)/ len(category_DCG), sum(NDCG) / len(NDCG))


In [69]:
#search for w
ws = np.linspace(0, 0.5, num = 21)
for w in ws:
    conformity, category_DCG, NDCG = cal_conformity_NDCG(dev_raw_response, rank_score, category_query_dev, category_gt_dev, w)
    print("w = %.3f, conformity = %.5f, category_DCG = %.5f, NDCG = %.5f " % (w, conformity, category_DCG, NDCG))

w = 0.000, conformity = 0.91346, category_DCG = 4.18978, NDCG = 0.50970 
w = 0.025, conformity = 0.91742, category_DCG = 4.20470, NDCG = 0.50801 
w = 0.050, conformity = 0.91870, category_DCG = 4.21035, NDCG = 0.50484 
w = 0.075, conformity = 0.91882, category_DCG = 4.21087, NDCG = 0.50189 
w = 0.100, conformity = 0.91845, category_DCG = 4.20888, NDCG = 0.49960 
w = 0.125, conformity = 0.91839, category_DCG = 4.20786, NDCG = 0.49800 
w = 0.150, conformity = 0.91815, category_DCG = 4.20619, NDCG = 0.49635 
w = 0.175, conformity = 0.91748, category_DCG = 4.20341, NDCG = 0.49539 
w = 0.200, conformity = 0.91650, category_DCG = 4.19870, NDCG = 0.49346 
w = 0.225, conformity = 0.91553, category_DCG = 4.19479, NDCG = 0.49177 
w = 0.250, conformity = 0.91480, category_DCG = 4.19108, NDCG = 0.49008 
w = 0.275, conformity = 0.91401, category_DCG = 4.18775, NDCG = 0.48917 
w = 0.300, conformity = 0.91315, category_DCG = 4.18354, NDCG = 0.48740 
w = 0.325, conformity = 0.91285, category_DCG = 4.1

In [70]:
conformity, category_DCG, NDCG = cal_conformity_NDCG(test_raw_response, rank_score, category_query_test, category_gt_test)
print("Test Set Result")
print("Baseline: w = %.3f, conformity = %.5f, category_DCG = %.5f, NDCG = %.5f " % (0.0, conformity, category_DCG, NDCG))
conformity, category_DCG, NDCG = cal_conformity_NDCG(test_raw_response, rank_score, category_query_test, category_gt_test, 0.075)
print("Rerank  : w = %.3f, conformity = %.5f, category_DCG = %.5f, NDCG = %.5f " % (0.075, conformity, category_DCG, NDCG))

Test Set Result
Baseline: w = 0.000, conformity = 0.92251, category_DCG = 4.21840, NDCG = 0.51849 
Rerank  : w = 0.075, conformity = 0.92709, category_DCG = 4.23684, NDCG = 0.51374 


0
1
2
3
4
5


In [98]:
data_folder

'/home/xz/JupyterNOtebooks/data'

In [105]:
with open(os.path.join(data_folder, 'gender', 'label', 'gender1.txt'), 'r') as f:
    gender0 = json.load(f)

In [108]:
a = pd.read_csv(os.path.join(data_folder, 'dev_query_w_predict.csv'))

In [110]:
a.loc[a['Query Set URL'] == 'https://s3-ap-southeast-1.amazonaws.com/visenze-productivity-evaluation/001207b0f0809bf78b1f8ba731a220130a7c7db0']

,Query Set URL,Category,Fq,Im_Name,Bounding Box,Box Categories,Tags,Customized Query Param,Ranking Tag,S3 URL,Image Id,category
1348,https://s3-ap-southeast-1.amazonaws.com/visenz...,shoe,NaN,NaN,"[[79,76,193,265]]",[shoe],Bad_Cases:exact_match_badcases,"{""detection"":""shoe""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2737642,shoe


In [112]:
a

,Query Set URL,Category,Fq,Im_Name,Bounding Box,Box Categories,Tags,Customized Query Param,Ranking Tag,S3 URL,Image Id,category
0,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[15,0,391,349]]",[bag],undefined:Bad Case,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734638,bag
1,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[29,15,261,396]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734640,bag
2,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[84,18,331,400]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734646,bag
3,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[0,24,1689,3023]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734661,bag
4,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[2,0,390,398]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734671,bag
5,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[60,0,379,394]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734675,bag
6,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[35,154,299,360]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734689,bag
7,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[40,4,358,378]]",[bag],Bad_Cases:bad_case_2017,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734692,bag
8,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[5,11,300,400]]",[bag],Bad_Cases:bad_case_2017,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734702,bag
9,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[76,9,311,290]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734705,bag
